<a href="https://colab.research.google.com/github/sugartastic81/CVdepthAI_OAK-D_TensionTerminator/blob/main/trainTTyolo8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tension Terminator Yolo Transferlearning

## Set global variables

In [1]:
# Set Variables
BASE_MODEL = "yolov8m"
TRAIN_EPOCHS = 100
TRAIN_SIZE = 320

TT_PROJECTNAME = "TensionTerminator"

## Mount Google Drive to backup files

In [2]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

google_drive_target_folder = '/content/drive/My Drive/'+TT_PROJECTNAME+'/'
# Check if the target folder exists in Google Drive, create if it doesn't
if not os.path.exists(google_drive_target_folder):
    os.makedirs(google_drive_target_folder)
    print(f"Created folder: {google_drive_target_folder}")
print("Google Drive Target Folder: " + google_drive_target_folder)

Mounted at /content/drive
Google Drive Target Folder: /content/drive/My Drive/TensionTerminator/


# Install Dependencies

In [3]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.6/646.6 kB 9.4 MB/s eta 0:00:00


In [4]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstalled opencv-python-headless-4.8.1.78
  Attempting uninstall: idna
    Found existin

## Python imports

In [5]:
import os
import random
import yaml
import shutil
from ultralytics import YOLO
from google.colab import userdata, files
from datetime import datetime
from ultralytics.utils.benchmarks import benchmark
from roboflow import Roboflow

## Loading dataset to train on

In [6]:
secret_key = userdata.get('rb-key')

rf = Roboflow(api_key=secret_key)
project = rf.workspace("juzaworkspace-wzpkn").project("tensionterminatordetectionv2")
dataset = project.version(5).download("yolov8")

DATASET_DIR = dataset.location
DATA_YAML = DATASET_DIR + "/data.yaml"
print("\n\info")
print("Data saved to: " + DATASET_DIR)
print("Data YAML: " + DATA_YAML)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.221, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to TensionTerminatorDetectionV2-5 in yolov8:: 100%|██████████| 2437/2437 [00:01<00:00, 1506.73it/s]



\info
Data saved to: /content/TensionTerminatorDetectionV2-5
Data YAML: /content/TensionTerminatorDetectionV2-5/data.yaml


There is a bug in the data.yaml of robfolow yolov8. This script corrects it.

In [7]:
# corrects the folders in the data.yaml

# Path to the YAML file
yaml_file_path = DATASET_DIR + 'data.yaml'

# Read the YAML file
with open(DATA_YAML) as file:
    data = yaml.safe_load(file)

# Update the paths
data['train'] = DATASET_DIR + '/train/images'
data['val'] = DATASET_DIR + '/valid/images'
data['test'] = DATASET_DIR + '/test/images'

# Write the updated data back to the YAML file
with open(DATA_YAML, 'w') as file:
    yaml.dump(data, file)

print("Updated data.yaml successfully.")


Updated data.yaml successfully.


## Train a new model

In [8]:
# Variables already set, but u can run to re-train just this cell and modify the vars (not recommended!)
#TRAIN_MODEL = "yolov8s"
#TRAIN_EPOCHS = 1
#TRAIN_SIZE = 320

# Load a yolo model as base
modelpt = BASE_MODEL + ".pt"
model = YOLO(modelpt)

# train the model
model.train(data=DATA_YAML, epochs=TRAIN_EPOCHS, imgsz=TRAIN_SIZE)

100%|██████████| 49.7M/49.7M [00:00<00:00, 354MB/s]


Ultralytics YOLOv8.0.221 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/TensionTerminatorDetectionV2-5/data.yaml, epochs=100, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_w

100%|██████████| 755k/755k [00:00<00:00, 45.2MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 186MB/s]


...

                   all         42         99       0.96      0.978      0.971      0.748
Stopping training early as no improvement observed in last 50 epochs. Best results observed at epoch 22, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



72 epochs completed in 0.760 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.221 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.09it/s]


                   all         42         99      0.981      0.967       0.99      0.772
              Duoballs         42         41      0.967      0.927       0.98      0.779
                  Head         42         41      0.988      0.976      0.995      0.845
     Triggerpointlever         42         17      0.988          1      0.995      0.693
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78ff3d8acf40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

## Save the model to Google Drive

In [9]:
# Get latest run dir
base_dir = '/content/runs/detect'
latest_run_dir = max([os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))], key=os.path.getmtime)
print(latest_run_dir)

# modify if there is an error
#latest_run_dir = "/content/runs/detect/train3"

# Define the base directory where the runs are stored
TRAINED_WEIGHTS = latest_run_dir + "/weights/" + "best.pt"

# Construct the path to the best weights file
print("Original trained weights path: ", TRAINED_WEIGHTS)

/content/runs/detect/train
Original trained weights path:  /content/runs/detect/train/weights/best.pt


In [10]:
# Get the current timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Construct the new file name with timestamp
new_file_name = f"TTmodel2_{BASE_MODEL}_img{TRAIN_SIZE}_{timestamp}.pt"
new_file_path = os.path.join(latest_run_dir, 'weights', new_file_name)

# Copy the file with timestamp in the same directory
shutil.copy2(TRAINED_WEIGHTS, new_file_path)
print("New file path with timestamp: ", new_file_path)

# Make sure the target folder exists
if not os.path.exists(google_drive_target_folder):
    os.makedirs(google_drive_target_folder)

# Construct the target file path in Google Drive
google_drive_target_path = os.path.join(google_drive_target_folder, new_file_name)

# Copy the file to Google Drive
shutil.copy2(new_file_path, google_drive_target_path)
print("Model file copied to Google Drive: ", google_drive_target_path)

New file path with timestamp:  /content/runs/detect/train/weights/TTmodel2_yolov8m_img320_20231130_213654.pt
Model file copied to Google Drive:  /content/drive/My Drive/TensionTerminator/TTmodel2_yolov8m_img320_20231130_213654.pt


## Evaluate Performance

In [11]:
# Evaluate the model's performance on the validation set
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.0.221 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/TensionTerminatorDetectionV2-5/valid/labels.cache... 42 images, 0 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.23s/it]


                   all         42         99      0.981      0.967       0.99      0.777
              Duoballs         42         41      0.967      0.927       0.98      0.773
                  Head         42         41      0.987      0.976      0.995      0.854
     Triggerpointlever         42         17      0.988          1      0.995      0.703
Speed: 0.1ms preprocess, 35.7ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to runs/detect/train2


array([    0.77265,     0.85429,      0.7027])

Testing on 3 random files

In [12]:
# Define the path to the test images directory
test_images_dir = DATASET_DIR + "/test/images"

# Get a list of all files in the test images directory
all_test_images = os.listdir(test_images_dir)

# Randomly select 3 images
random_test_images = random.sample(all_test_images, 3)

# Predict on each of the 3 random images
for image_name in random_test_images:
    image_path = os.path.join(test_images_dir, image_name)
    results = model(image_path)  # Perform prediction
    print(f"Results for {image_name}:")
    print(results)  # or use results.show() or results.save() based on your requirement


image 1/1 /content/TensionTerminatorDetectionV2-5/test/images/videos-2023-08-12-01_31_00-071237_000120_0204_png.rf.52cf2ae2986fc03d6498ad7b06555f36.jpg: 256x320 1 Duoballs, 1 Head, 94.8ms
Speed: 1.6ms preprocess, 94.8ms inference, 2.1ms postprocess per image at shape (1, 3, 256, 320)
Results for videos-2023-08-12-01_31_00-071237_000120_0204_png.rf.52cf2ae2986fc03d6498ad7b06555f36.jpg:
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Duoballs', 1: 'Head', 2: 'Triggerpointlever'}
orig_img: array([[[155, 133, 121],
        [155, 133, 121],
        [155, 133, 121],
        ...,
        [207, 220, 234],
        [205, 218, 232],
        [203, 216, 230]],

       [[152, 130, 118],
        [153, 131, 119],
        [154, 132, 120],
        ...,
        [208, 221, 235],
        [206, 219, 233],
        [204, 217, 231]],

       [[151, 129, 117],
        [151, 129, 117],
        [152, 130, 118],
  

## Export the file to OAK-D

In [13]:
print("\nGo to Tool at http://tools.luxonis.com/ and make a export of the downloaded model file " + new_file_name + " for the OAK-Camera.")
print("\nPro Tipps to use the Luxonis tool: \n  *) choose the correct Yolo Version (example: YoloV8 detection only)\n  *) choose the correct image shape (example: "+ str(TRAIN_SIZE) + ")\n  *) edit the advanced options, choose 6 shaves and deactivate OpenVINO 2021.4 checkbox.\n\n")

files.download(new_file_path)


Go to Tool at http://tools.luxonis.com/ and make a export of the downloaded model file TTmodel2_yolov8m_img320_20231130_213654.pt for the OAK-Camera.

Pro Tipps to use the Luxonis tool: 
  *) choose the correct Yolo Version (example: YoloV8 detection only)
  *) choose the correct image shape (example: 320)
  *) edit the advanced options, choose 6 shaves and deactivate OpenVINO 2021.4 checkbox.




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
# for other use cases u can follow the ONNX path https://docs.luxonis.com/en/latest/pages/model_conversion/#converting-model-to-myriadx-blob
# converts the model to onnx
'''
path = model.export(format="onnx")  # export the model to ONNX format
TRAINED_WEIGHTS_ONNX = latest_run_dir + "/weights/" + "best.onnx"
files.download(TRAINED_WEIGHTS_ONNX)
'''

'\npath = model.export(format="onnx")  # export the model to ONNX format\nTRAINED_WEIGHTS_ONNX = latest_run_dir + "/weights/" + "best.onnx"\nfiles.download(TRAINED_WEIGHTS_ONNX)\n'

In [15]:
# Benchmark on GPU
# deactivated, cause needs a lot of time

benchmark(model=TRAINED_WEIGHTS, data=DATA_YAML, imgsz=TRAIN_SIZE, half=False, device=0)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.0/78.2 GB disk)

Benchmarks complete for best.pt on /content/TensionTerminatorDetectionV2-5/data.yaml at imgsz=320 (408.29s)
                   Format Status❔  Size (MB)  metrics/mAP50-95(B)  Inference time (ms/im)
0                 PyTorch       ✅       49.6               0.7765                   15.23
1             TorchScript       ✅       99.1               0.7702                   14.24
2                    ONNX       ✅       98.7               0.7702                  226.38
3                OpenVINO       ❌        0.0                  NaN                     NaN
4                TensorRT       ✅      107.0               0.7702                   12.21
5                  CoreML       ❌        0.0                  NaN                     NaN
6   TensorFlow SavedModel       ❌        0.0                  NaN                     NaN
7     TensorFlow GraphDef       ❌        0.0                  NaN                     NaN
8         TensorFlow Li

,Format,Status❔,Size (MB),metrics/mAP50-95(B),Inference time (ms/im)
0,PyTorch,✅,49.6,0.7765,15.23
1,TorchScript,✅,99.1,0.7702,14.24
2,ONNX,✅,98.7,0.7702,226.38
3,OpenVINO,❌,0.0,NaN,NaN
4,TensorRT,✅,107.0,0.7702,12.21
5,CoreML,❌,0.0,NaN,NaN
6,TensorFlow SavedModel,❌,0.0,NaN,NaN
7,TensorFlow GraphDef,❌,0.0,NaN,NaN
8,TensorFlow Lite,❌,0.0,NaN,NaN
9,TensorFlow Edge TPU,❌,0.0,NaN,NaN


In [16]:
# https://docs.ultralytics.com/guides/hyperparameter-tuning/#usage-example

# END